<a href="https://colab.research.google.com/github/EffiSciencesResearch/ML4G-2.0/blob/master/workshops/evals/evals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
# Evals

## Contents

- 
- 
..

## Goals

- Get experience using Inspect, the open source evals framework from UK AI Safety Institute.
- Get experience conducting evals.
- Hard version of notebook: doing various 'side tasks' that research can involve, e.g. exploring a new code base, finding the history of a research idea, etc.

## Setup

In [ ]:
try:
    import google.colab

    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    %pip install inspect-ai openai
else:
    !pip install inspect-ai openai

In [ ]:
import os
from inspect_ai import Task, task, eval
from inspect_ai.dataset import example_dataset
from inspect_ai.scorer import model_graded_fact
from inspect_ai.solver import chain_of_thought, generate, self_critique

from inspect_ai.log._log import EvalSample, EvalLog
from inspect_ai.model._chat_message import ChatMessageUser, ChatMessageAssistant

In [ ]:
# Replace `your-api-key` with an actual openai api key.
os.environ["OPENAI_API_KEY"] = "your-api-key"

## 1. Running a simple eval in Inspect

The first exercise introduces the main components of the Inspect framework:

- Datasets
  - These are sets of (labeled) samples.
A vanilla dataset would consist of a list of input-target pairs, where inputs are prompts to be given to an LLM and targets somehow indicate what an LLM's output should be (or should not, in the case of detecting unwanted behaviour).
- Plans and solvers
  - The core solver is `generate()` which gets the LLM to generate text based on the current message history.
In the simplest case, the message history will consist of one user message containing the input from the dataset sample.
Other solvers can add scaffolding by changing the message history, e.g. so that the LLM uses chain of thought.
A plan is a sequence of solvers that provide the full scaffolding for the LLM.
- Scorers
  - These score the outputs of the LLMs, often involving some kind of comparison with the 'target'.
You can do hard-coded scoring (e.g. using regex to detect answer to a multiple choice question) or use another LLM to do the scoring.
- Tasks
  - A task is a combination of a dataset, plan and a scorer. A task is then run on an LLM you want to evaluate.

The example below is based on Inspect's documentation https://inspect.ai-safety-institute.org.uk/#getting-started and https://inspect.ai-safety-institute.org.uk/workflow.html.
Some adjustments are made to work in a (colab) notebook - Inspect is designed to be *run* via a CLI but they do recommend using a notebook while exploring and developing eval workflows.

### Datasets

Inspect includes some in-built datasets.
We will use a dataset that evaluates an LLMs understanding of theory of mind.

You can view the raw jsonl file where the dataset is stored on Inspect's GitHub repo at the link
- https://github.com/UKGovernmentBEIS/inspect_ai/blob/main/src/inspect_ai/dataset/_examples/theory_of_mind.jsonl

For those interested in the history of ideas, this dataset's history can be traced through these three repos:
- https://github.com/openai/evals/tree/main/evals/registry/data/theory_of_mind/tomi
- https://github.com/facebookresearch/ToMi
- https://github.com/kayburns/tom-qa-dataset

In [ ]:
dataset = example_dataset("theory_of_mind")
print(len(dataset))

We see this dataset has 100 samples.
Let us look at an individual sample.

In [ ]:
sample = dataset[2]

# print the raw input string for this sample
# Hide: hard
print(sample.input[0].content)
print()
# Hide: none

# print the answer string for this sample
# Hide: hard
print(sample.target)
# Hide: none

# Hide: all
5
# Hide: none

From the above, we see that this dataset is straightforward, and is essentially a list of 100 question-answer pairs.

### Plans and solvers

Appropriate text

Appropriate text

In [ ]:
DEFAULT_COT_TEMPLATE = r"""
{prompt}

Before answering, reason in a step-by-step manner as to get the right answer. Provide your answer at the end on its own line in the form "ANSWER: $ANSWER" (without quotes) where $ANSWER is the answer to the question.
"""

Appropriate text

In [ ]:
DEFAULT_CRITIQUE_TEMPLATE = r"""
Given the following question and answer, please critique the answer. A good answer comprehensively answers the question and NEVER refuses to answer. If the answer is already correct do not provide critique - simply respond 'The original answer is fully correct'.

[BEGIN DATA]
***
[Question]: {question}
***
[Answer]: {completion}
***
[END DATA]

Critique: """


DEFAULT_CRITIQUE_COMPLETION_TEMPLATE = r"""
Given the following question, initial answer and critique please generate an improved answer to the question:

[BEGIN DATA]
***
[Question]: {question}
***
[Answer]: {completion}
***
[Critique]: {critique}
***
[END DATA]

If the original answer is already correct, just repeat the original answer exactly. Provide your answer at the end on its own line in the form "ANSWER: $ANSWER" (without quotes) where $ANSWER is the answer to the question.
"""

### Scorers

Appropriate text

In [ ]:
scorer = model_graded_fact(model="openai/gpt-4o-mini")

In [ ]:
DEFAULT_MODEL_GRADED_FACT_TEMPLATE = """
You are comparing a submitted answer to an expert answer on a given question. Here is the data:

[BEGIN DATA]
************
[Question]: {question}
************
[Expert]: {criterion}
************
[Submission]: {answer}
************
[END DATA]

Compare the factual content of the submitted answer with the expert answer. Ignore any differences in style, grammar, or punctuation.

Does the submission contain the content in the expert answer?

{instructions}
"""

DEFAULT_INSTRUCTIONS = """
After assessing the submitted answer, reply with 'GRADE: $LETTER' (without quotes) where LETTER is one of C{partial_letter}I.  Please choose ONE option for the grade: either "C" for correct answers, {partial_prompt}or "I" for incorrect answers.

For example, after reviewing a correct answer you might write 'GRADE: C' or after reviewing an incorrect answer you might write 'GRADE: I'.

First, write out in a step by step manner your reasoning about the criterion to be sure that your conclusion is correct. Avoid simply stating the correct answers at the outset. Then, end with your answer formatted as 'GRADE: $LETTER' (without quotes) where LETTER is one of C{partial_letter}I.
"""

### Running the evals

Appropriate text

In [ ]:
@task
def theory_of_mind(model: str):
    return Task(
        dataset=dataset[0:3],
        plan=[
            chain_of_thought(),
            generate(),
            self_critique(model=model),
        ],
        scorer=scorer,
    )

In [ ]:
model = "openai/gpt-4o-mini"
logs = eval(
    theory_of_mind(model),
    model=model,
)

In [ ]:
def print_info_from_sample(sample: EvalSample):
    print("---MESSAGE HISTORY---")
    for message in sample.messages:
        if isinstance(message, ChatMessageUser):
            print(f"User: {message.content}" + "\n")
        elif isinstance(message, ChatMessageAssistant):
            print(f"Assistant: {message.content}" + "\n")

    print("--SCORER INFORMATION---")
    value = sample.scores["model_graded_fact"].value
    print(f"Score: {'correct' if value=='C' else 'incorrect'}")
    print(f"Explanation: {sample.scores['model_graded_fact'].explanation}")


def print_info_from_logs(logs: list[EvalLog]):
    for i, sample in enumerate(logs[0].samples):
        print(f"===START OF SAMPLE {i}===")
        print_info_from_sample(sample)
        print(f"===END OF SAMPLE {i}===")

In [ ]:
print_info_from_logs(logs)